In [6]:
import time
import pandas as pd
import requests as re
from bs4 import BeautifulSoup as bs

In [7]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
"HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
"MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
"NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
"SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [8]:
t1 = time.time()
df = pd.DataFrame(columns = ['park_state','park_name','park_type','park_loc','park_desc'])
for state in states:
    url = "https://www.nps.gov/state/%s/index.htm" % state
    result = re.get(url)
    soup = bs(result.content,"lxml")
    parks = soup.find_all("li","clearfix")
    for park in parks[:-1]:
        park_name = park.find('h3').text
        park_type = park.find('h2').text
        park_loc = park.find('h4').text
        park_desc = park.find('p').text

        df.loc[len(df)] =  [state, park_name, park_type, park_loc, park_desc]
print("Time to retrieve: %0.3f seconds" % (time.time() - t1))

Time to retrieve: 36.717 seconds


In [9]:
df.head()

,park_state,park_name,park_type,park_loc,park_desc
0,AL,Birmingham Civil Rights,National Monument,AL,"\nIn 1963, images of snarling police dogs unle..."
1,AL,Freedom Riders,National Monument,AL,"\nIn 1961, a small interracial band of “Freedo..."
2,AL,Horseshoe Bend,National Military Park,"Daviston, AL","\nOn 27 March 1814, Major General Andrew Jacks..."
3,AL,Little River Canyon,National Preserve,"Fort Payne, AL",\nLittle River is unique because it flows for ...
4,AL,Muscle Shoals,National Heritage Area,,\nThe Tennessee River brought the early Native...


In [ ]:
df.tail()

In [10]:
df["park_lat"] = ""
df["park_long"] = ""

In [11]:
df.head()

,park_state,park_name,park_type,park_loc,park_desc,park_lat,park_long
0,AL,Birmingham Civil Rights,National Monument,AL,"\nIn 1963, images of snarling police dogs unle...",,
1,AL,Freedom Riders,National Monument,AL,"\nIn 1961, a small interracial band of “Freedo...",,
2,AL,Horseshoe Bend,National Military Park,"Daviston, AL","\nOn 27 March 1814, Major General Andrew Jacks...",,
3,AL,Little River Canyon,National Preserve,"Fort Payne, AL",\nLittle River is unique because it flows for ...,,
4,AL,Muscle Shoals,National Heritage Area,,\nThe Tennessee River brought the early Native...,,


In [13]:
t1 = time.time()
for i in range(0,len(df)-1):
    park_search = (df.park_name[i] + ' ' + df.park_type[i]).replace(' ','+')
    park_search += '+coordinates'

    url = "https://www.google.com/search?q=%s" % park_search
    result = re.get(url)
    soup = bs(result.content,"lxml")
    
    try:
        spans = soup.find_all('span', attrs={'class':'mrH1y'})
        park_coords = spans[0].string

        park_coords =park_coords.replace(" ","").split(',')
        lat, long = park_coords[0],park_coords[1]
        lat = lat.split("°")
        long = long.split("°")

        park_lat= float(lat[0])
        if(lat[1] == 'S'):
            park_lat = park_lat * -1

        park_long = float(long[0])
        if(long[1] == 'W'):
            park_long = park_long * -1

        df.park_lat[i] = park_lat
        df.park_long[i] = park_long
    except:
        pass

print("Time to retrieve: %0.3f seconds" % (time.time() - t1))

Time to retrieve: 878.044 seconds


In [15]:
df.head(10)

,park_state,park_name,park_type,park_loc,park_desc,park_lat,park_long
0,AL,Birmingham Civil Rights,National Monument,AL,"\nIn 1963, images of snarling police dogs unle...",33.5153,-86.8146
1,AL,Freedom Riders,National Monument,AL,"\nIn 1961, a small interracial band of “Freedo...",33.6351,-85.9085
2,AL,Horseshoe Bend,National Military Park,"Daviston, AL","\nOn 27 March 1814, Major General Andrew Jacks...",32.9783,-85.731
3,AL,Little River Canyon,National Preserve,"Fort Payne, AL",\nLittle River is unique because it flows for ...,34.3884,-85.6254
4,AL,Muscle Shoals,National Heritage Area,,\nThe Tennessee River brought the early Native...,,
5,AL,Natchez Trace,Parkway,"the states of AL,MS,TN",\nThe Natchez Trace Parkway is a 444-mile recr...,33.0578,-89.589
6,AL,Natchez Trace,National Scenic Trail,"Tupelo, AL,MS,TN",\nThe 450-mile foot trail that became known as...,,
7,AL,Russell Cave,National Monument,"Bridgeport, AL",\nRussell Cave is an archaeological site with ...,34.9765,-85.8143
8,AL,Selma To Montgomery,National Historic Trail,"Montgomery, Lowndes & Dallas Counties, AL","\nOn August 6, 1965, President Lyndon Johnson ...",,
9,AL,Trail Of Tears,National Historic Trail,"AL,AR,GA,IL,KY,MO,NC,OK,TN",\nRemember and commemorate the survival of the...,,


In [17]:
df.to_excel("C:\\users\\tplobbregt\\desktop\\df.xlsx")